<a href="https://colab.research.google.com/github/mihiarc/global-timber/blob/main/Forest_Area_by_Country.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Global Forest Area by Country

Summarize total forest area for Sweden using Dynamic World v1 derived from Setinel 2 data.

In [1]:
import geemap
import ee
ee.Authenticate()
ee.Initialize(project='ee-chrismihiar')

In [2]:
# GAUL country boundaries
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
Estonia = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Estonia'))
Sweden = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Sweden'))
France = gaul.filter(ee.Filter.eq('ADM0_NAME', 'France'))

In [3]:
startDate = '2020-01-01'
endDate = '2021-01-01'
geometry = France.geometry()

In [4]:
Map = geemap.Map()
Map.centerObject(geometry, 6)
Map

Map(center=[46.513593004816066, 2.5745098025265016], controls=(WidgetControl(options=['position', 'transparent…

In [5]:
dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')\
            .filterDate(startDate, endDate)\
            .filterBounds(geometry)

# Create a mode composite.
classification = dw.select('label')
dwComposite = classification.reduce(ee.Reducer.mode())

# Extract the trees class.
treesArea = dwComposite.eq(1)

In [6]:
dwVisParams = {
  'min': 0,
  'max': 8,
  'palette': [
    '#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635', '#DFC35A',
    '#C4281B', '#A59B8F', '#B39FE1'
  ]
}

#Clip the composite and add it to the Map.
Map.addLayer(dwComposite.clip(geometry), dwVisParams, 'Classified Composite')
Map.addLayer(treesArea.clip(geometry), {}, 'Trees')
Map

Map(bottom=6095.0, center=[46.513593004816066, 2.5745098025265016], controls=(WidgetControl(options=['position…

In [7]:
dwComposite = dwComposite.rename(['classification'])
treesArea = treesArea.rename(['Trees'])

In [8]:
#Count all pixels.
statsTotal = treesArea.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=geometry,
    scale=10,
    maxPixels=1e10)
totalPixels = statsTotal.get('Trees')

#Mask 0 pixel values and count remaining pixels.
treesAreaMasked = treesArea.selfMask()

statsMasked = treesAreaMasked.reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=geometry,
    scale=10,
    maxPixels=1e10)
treesAreaPixels = statsMasked.get('Trees')
# print('Tree area pixel count:')
# treesAreaPixels.getInfo() # too big to print to console

In [9]:
fraction = (ee.Number(treesAreaPixels).divide(totalPixels)) \
  .multiply(100)
print('Percentage tree area:')
fraction.format('%.2f')

Percentage tree area:


In [ ]:
treesAreaInSqKm = ee.Number(treesAreaPixels).divide(1e6).round()
print('Tree area in square kilometers:')
treesAreaInSqKm.getInfo()

In [ ]:
print('Total Area in square kilometers:')
areaInSqKm = ee.Number(totalPixels).divide(1e6).round()
areaInSqKm.getInfo()

In [ ]:
# count pixels for all classes
pixelCountStats = dwComposite.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram().unweighted(),
    geometry=geometry,
    scale=10,
    maxPixels=1e10)

classLabels = ee.List([
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'
    ])

#Rename keys with class names.
pixelCounts = ee.Dictionary(pixelCountStats.get('classification'))
pixelCountsFormatted = pixelCounts.rename(
  pixelCounts.keys(), classLabels)
pixelCountsFormatted.getInfo()

## Calculate zonal stats for one image

In [ ]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
stats = os.path.join(out_dir, "stats.csv")

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Get the most likely class probability.
# top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

geemap.zonal_stats(dw_image.select('trees'), country,
                            stats,
                            statistics_type='MEAN',
                            decimal_places=2,)
geemap.create_download_link(stats)

In [ ]:
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
out_landcover_stats = os.path.join(out_dir, "landcover_wake_nc.csv")

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

geemap.zonal_stats_by_group(dwComposite, filtered, out_landcover_stats, scale=1000)
geemap.create_download_link(out_landcover_stats)

In [ ]:
print(type(dwComposite))
print(type(builtArea))

In [ ]:
# Rename the band names.
dwComposite = dwComposite.rename(['classification'])
builtArea = builtArea.rename(['built_area'])

# Calculate Pixel Counts.

# Count all pixels.
statsTotal = builtArea.reduceRegion({
    'reducer': ee.Reducer.count(),
    'geometry': geometry,
    'scale': 10,
    'maxPixels': 1e10
    })
totalPixels = statsTotal.get('built_area')

In [ ]:
m

In [ ]:
from ee_plugin import Map

# Calculating Statistics of a Region

# Estimate what percentage of a region is covered by built-up
# areas in a county.

# Start by creating a mode composite for the given time period.
counties = ee.FeatureCollection('TIGER/2016/Counties')
filtered = counties.filter(ee.Filter.eq('NAMELSAD', 'Dane County'))
geometry = filtered.geometry()
m.centerObject(geometry, 10)

startDate = '2020-01-01'
endDate = '2021-01-01'

dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
  .filterDate(startDate, endDate) \
  .filterBounds(geometry)

# Create a Mode Composite.
classification = dw.select('label')
dwComposite = classification.reduce(ee.Reducer.mode())

# Extract the Built Area class.
builtArea = dwComposite.eq(6)

dwVisParams = {
  'min': 0,
  'max': 8,
  'palette': ['#419BDF', '#397D49', '#88B053', '#7A87C6',
    '#E49635', '#DFC35A', '#C4281B', '#A59B8F', '#B39FE1']
}

# Clip the composite and add it to the m.
m.addLayer(dwComposite.clip(geometry), dwVisParams, 'Classified Composite')
m.addLayer(builtArea.clip(geometry), {}, 'Built Areas')

# Rename the band names.
dwComposite = dwComposite.rename(['classification'])
builtArea = builtArea.rename(['built_area'])

# Calculate Pixel Counts.

# Count all pixels.
statsTotal = builtArea.reduceRegion({
    'reducer': ee.Reducer.count(),
    'geometry': geometry,
    'scale': 10,
    'maxPixels': 1e10
    })
totalPixels = statsTotal.get('built_area')

# Mask 0 pixel values and count remaining pixels.
builtAreaMasked = builtArea.selfMask()

statsMasked = builtAreaMasked.reduceRegion({
    'reducer': ee.Reducer.count(),
    'geometry': geometry,
    'scale': 10,
    'maxPixels': 1e10
    })
builtAreaPixels = statsMasked.get('built_area')
print('Built area pixel count', builtAreaPixels)
fraction = (ee.Number(builtAreaPixels).divide(totalPixels)) \
  .multiply(100)
print('Percentage built area', fraction.format('%.2f'))

# Calculating Pixel Counts for all classes.
pixelCountStats = dwComposite.reduceRegion({
    'reducer': ee.Reducer.frequencyHistogram().unweighted(),
    'geometry': geometry,
    'scale': 10,
    'maxPixels': 1e10
    })

pixelCounts = ee.Dictionary(pixelCountStats.get('classification'))
print('Pixel counts per class', pixelCounts)

# Format the results to make it more readable.
classLabels = ee.List([
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'
    ])

# Rename keys with class names.
pixelCountsFormatted = pixelCounts.rename(
  pixelCounts.keys(), classLabels)
print('Pixel counts per class (formatted)', pixelCountsFormatted)

# Create a Feature Collection.
exportFc = ee.FeatureCollection(
  ee.Feature(None, pixelCountsFormatted))

# Export the results as a CSV file.
Export.table.toDrive({
  'collection': exportFc,
  'description': 'pixel_counts_export',
  'folder': 'earthengine',
  'fileNamePrefix': 'pixel_counts',
  'fileFormat': 'CSV',
})

m